In [ ]:
# Import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import json
import os

# Install required packages if not already installed
# !pip install tensorflow mediapipe

# Dataset paths (same as your previous implementation)
train_path = r"C:\Users\thula\Documents\Mini_Project\Dataset\training_set"
test_path = r"C:\Users\thula\Documents\Mini_Project\Dataset\test_set"

# Image parameters (matching your existing setup)
IMG_SIZE = 64  # Maintain 64x64 to match your dataset
BATCH_SIZE = 32
NUM_CLASSES = 26  # A-Z classes

# Data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,  # Important for sign language
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create data flows
train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Save class indices
with open("class_indices.json", "w") as f:
    json.dump(train_set.class_indices, f)

# Create ResNet50V2 base model
base_model = ResNet50V2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Compile model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-7
)

# Train the model
""""history = model.fit(
    train_set,
    steps_per_epoch=200 // BATCH_SIZE,
    validation_data=test_set,
    validation_steps=50,
    epochs=30,
    callbacks=[early_stop, reduce_lr]
)"""""
EPOCHS = 10
STEPS_PER_EPOCH = 120  # Full training
VALIDATION_STEPS = test_set.samples // BATCH_SIZE  # Full validation

history = model.fit(
    train_set,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=test_set,
    validation_steps=VALIDATION_STEPS,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr]
)

# Save model
model.save('asl_resnet50v2_model.h5')

# Plot training history


# Evaluate on test set
loss, accuracy = model.evaluate(test_set)
print(f"\nTest Accuracy: {accuracy*100:.2f}%")


In [2]:
loss, accuracy = model.evaluate(test_set)
print(f"\nTest Accuracy: {accuracy*100:.2f}%")

1/1 [==============================] - 2s 2s/step - loss: 0.7608 - accuracy: 0.8077

Test Accuracy: 80.77%
